#### Notes

Running the script below to create events for 2022-2023 academic calendar ([link](https://docs.google.com/spreadsheets/d/1a0ov5ctf7XA8DpAWxyScy99tJVCp90GKvwv_mmWbIHQ/edit#gid=1480204723))

In [1]:
from __future__ import print_function
import datetime
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import numpy as np
import os.path
import pickle

In [2]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [3]:
print(datetime.datetime.today())

2023-08-02 20:48:33.857741


<IPython.core.display.Javascript object>

In [4]:
creds = None
# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/calendar',
          'https://www.googleapis.com/auth/spreadsheets.readonly'
]

<IPython.core.display.Javascript object>

In [5]:
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=920945310291-dor04bbrppc1m1642g4bhrqr4734ev4s.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A58234%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fspreadsheets.readonly&state=2ElT4UCh0moPfvsQXphw9wlosQiGyc&access_type=offline


<IPython.core.display.Javascript object>

In [6]:
# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/calendar',
          'https://www.googleapis.com/auth/spreadsheets.readonly'
]

calendarID="c_822m07aktpjne3gfbr7cj8gjks@group.calendar.google.com"

"""Shows basic usage of the Google Calendar API.
Prints the start and name of the next 10 events on the user's calendar.
"""
creds = None
# The file token.pickle stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.

if os.path.exists('token.pickle'):
    print("Reading token...")
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)

service = build('calendar', 'v3', credentials=creds)

Reading token...


<IPython.core.display.Javascript object>

In [7]:
#service.events().list(calendarId=calendarID).execute()

<IPython.core.display.Javascript object>

In [8]:
#creds

<IPython.core.display.Javascript object>

In [9]:
# Call the Calendar API
now = datetime.datetime.utcnow().isoformat() + 'Z' # 'Z' indicates UTC time
print('Getting the upcoming 10 events')
events_result = service.events().list(calendarId=calendarID, timeMin=now,
                                    maxResults=10, singleEvents=True,
                                    orderBy='startTime').execute()
events = events_result.get('items', [])

if not events:
    print('No upcoming events found.')
for event in events:
    start = event['start'].get('dateTime', event['start'].get('date'))
    print(start, event['summary'])


Getting the upcoming 10 events
2023-08-19T11:00:00-07:00 New Student Orientation


<IPython.core.display.Javascript object>

### Pull data from Google sheet

In [10]:
# obtained from google sheet url:
# https://docs.google.com/spreadsheets/d/1jy2kZF4rHHUcGEzYcZfA6kukUdqaSj7FWzKIoyoapcM/edit#gid=0
SPREADSHEET_ID = "1jy2kZF4rHHUcGEzYcZfA6kukUdqaSj7FWzKIoyoapcM"

<IPython.core.display.Javascript object>

In [11]:
from googleapiclient.discovery import build
def pull_sheet_data(SPREADSHEET_ID, RANGE_NAME):
    
    service = build('sheets', 'v4', credentials=creds)
    sheet = service.spreadsheets()
    result = sheet.values().get(
        spreadsheetId=SPREADSHEET_ID,
        range=RANGE_NAME).execute()
    values = result.get('values', [])
    
    if not values:
        print('No data found.')
    else:
        rows = sheet.values().get(spreadsheetId=SPREADSHEET_ID,
                                  range=RANGE_NAME).execute()
        data = rows.get('values')
        print("COMPLETE: Data copied")
        return data
data = pull_sheet_data(SPREADSHEET_ID , 'Computer_Readable!A1:G')

COMPLETE: Data copied


<IPython.core.display.Javascript object>

In [12]:
# sanity check 
print(len(data))
data[1]

196


['08/19/2023',
 '11 am - 1 pm',
 'new students',
 'students2026',
 'New Student Orientation',
 '1']

<IPython.core.display.Javascript object>

In [14]:
import pandas as pd
df = pd.DataFrame(data[1:], columns=data[0])
df['start']=df.Time.str.split(' - ').map(lambda x: x[0])
df['end']=df.Time.str.split(' - ').map(lambda x: x[1])

print(df.shape)
df.head(2)

(195, 8)


,Date,Time,Calendar Assignment,Share with group,Event Title,Color,start,end
0,08/19/2023,11 am - 1 pm,new students,students2026,New Student Orientation,1,11 am,1 pm
1,08/26/2023,1 pm - 3:30 pm,volunteers,volunteers,Volunteer Orientation,1,1 pm,3:30 pm


<IPython.core.display.Javascript object>

In [15]:
if "calendarEventID" in df.columns:
    df["calendarEventID"] = df["calendarEventID"].replace(" ", None)
else:
    df["calendarEventID"] = None

<IPython.core.display.Javascript object>

In [16]:
df.head()

,Date,Time,Calendar Assignment,Share with group,Event Title,Color,start,end,calendarEventID
0,08/19/2023,11 am - 1 pm,new students,students2026,New Student Orientation,1,11 am,1 pm,None
1,08/26/2023,1 pm - 3:30 pm,volunteers,volunteers,Volunteer Orientation,1,1 pm,3:30 pm,None
2,09/09/2023,10 am - 2 pm,ec,ec,Kick-off,1,10 am,2 pm,None
3,09/09/2023,10 am - 2 pm,WCT,wct-instructors,Kick-off,1,10 am,2 pm,None
4,09/09/2023,10 am - 2 pm,Math Matters,math-instructors,Kick-off,1,10 am,2 pm,None


<IPython.core.display.Javascript object>

In [32]:
df.iloc[-1]["Color"] = 1

<IPython.core.display.Javascript object>

In [17]:
eventservice=service.events()

<IPython.core.display.Javascript object>

In [34]:
import time
for k,row in df.iterrows():
    try:
        starttime=datetime.datetime.strptime(row.Date + " " + row.start.strip(), '%m/%d/%Y %I:%M %p')
    except ValueError:
        starttime=datetime.datetime.strptime(row.Date + " " + row.start.strip(), '%m/%d/%Y %I %p')
        
    try:
        endtime=datetime.datetime.strptime(row.Date + " " + row.end.strip(), '%m/%d/%Y %I:%M %p')
    except ValueError:
        endtime=datetime.datetime.strptime(row.Date + " " + row.end.strip(), '%m/%d/%Y %I %p')
    
    body={
        "description": row['Event Title'],
        "summary":  row['Event Title'],
        "start":{
            "dateTime": starttime.isoformat(),
            "timeZone": "America/Los_Angeles"
        },
        "end":{
            "dateTime": endtime.isoformat(),
            "timeZone": "America/Los_Angeles"
        },
        "colorId": f"{row.Color}",
        "guestsCanInviteOthers": False

    }
    
    
    if row.calendarEventID:
        pass
    else:
        r=eventservice.insert(calendarId=calendarID,
                             body=body)
        response=r.execute()
        df.loc[k, 'calendarEventID']=response["id"]
        time.sleep(.2)

<IPython.core.display.Javascript object>

In [35]:
df.tail(2)

,Date,Time,Calendar Assignment,Share with group,Event Title,Color,start,end,calendarEventID
193,05/11/2024,10 am - 1 pm,ec,active,Graduation,1,10 am,1 pm,ka8t400qkver1hr62nd450j0uo
194,06/08/2024,11 am - 1 pm,ec,ec,EC EOY Retreat,1,11 am,1 pm,02s9cbargn8enaksrb9timbpi0


<IPython.core.display.Javascript object>

In [ ]:
TEST = "08/19/23 11 am"
starttime=datetime.datetime.strptime(TEST, '%m/%d/%y %I:%M %p')

In [ ]:
TEST = "8/26/2023 1 pm"
starttime=datetime.datetime.strptime(TEST, '%m/%d/%y %I %p')
starttime

In [36]:
df.head()

,Date,Time,Calendar Assignment,Share with group,Event Title,Color,start,end,calendarEventID
0,08/19/2023,11 am - 1 pm,new students,students2026,New Student Orientation,1,11 am,1 pm,p20kvodiu13fp9opogjctd5eds
1,08/26/2023,1 pm - 3:30 pm,volunteers,volunteers,Volunteer Orientation,1,1 pm,3:30 pm,qa5bogl8i12og8c52mjp890si8
2,09/09/2023,10 am - 2 pm,ec,ec,Kick-off,1,10 am,2 pm,b33uhacdr8glocr344f1egge3s
3,09/09/2023,10 am - 2 pm,WCT,wct-instructors,Kick-off,1,10 am,2 pm,f998sjl6ghr9muqj4fjrddqo54
4,09/09/2023,10 am - 2 pm,Math Matters,math-instructors,Kick-off,1,10 am,2 pm,ca6q7ju0qm13o28tntev47edao


<IPython.core.display.Javascript object>

In [37]:
for k,row in df.iterrows():
    emails=map(lambda x: x.replace(' ','') ,row['Share with group'].split(','))
    event = eventservice.get(calendarId=calendarID, eventId=row.calendarEventID).execute()
    attendees=event.get('attendees',[])
    for email in emails:
        try:
            invite=next(a for a in attendees if a['email']==email + "@mindsmatterseattle.org")
        except StopIteration:
            print(f'inviting {email} to {event["summary"]}')
            invite = {
                'email': f'{email}@mindsmatterseattle.org'
            }
            attendees.append(invite)

    event['attendees'] =  attendees 
    eventservice.update(calendarId=calendarID, eventId=row.calendarEventID, body=event).execute()
   

inviting students2026 to New Student Orientation
inviting volunteers to Volunteer Orientation
inviting ec to Kick-off
inviting wct-instructors to Kick-off
inviting math-instructors to Kick-off
inviting testprep-instructors to Kick-off
inviting senior-enrichment-instructors to Kick-off
inviting 2026mentors to Kick-off
inviting students2026 to Kick-off
inviting 2025mentors to Kick-off
inviting students2025 to Kick-off
inviting 2024mentors to Kick-off
inviting students2024 to Kick-off
inviting wct-instructors to Writing and Critical Thinking
inviting students2026 to Writing and Critical Thinking
inviting math-instructors to Math Matters
inviting students2026 to Math Matters
inviting testprep-instructors to Test Prep
inviting students2025 to Test Prep
inviting senior-enrichment-instructors to Senior Enrichment
inviting students2024 to Senior Enrichment
inviting 2026mentors to Sophomore Mentoring
inviting students2026 to Sophomore Mentoring
inviting 2025mentors to Junior Mentoring
inviting 

<IPython.core.display.Javascript object>

In [38]:
df.to_csv('2023-2024_calendar_events.csv', index=False)

<IPython.core.display.Javascript object>

In [39]:
df.head()

,Date,Time,Calendar Assignment,Share with group,Event Title,Color,start,end,calendarEventID
0,08/19/2023,11 am - 1 pm,new students,students2026,New Student Orientation,1,11 am,1 pm,p20kvodiu13fp9opogjctd5eds
1,08/26/2023,1 pm - 3:30 pm,volunteers,volunteers,Volunteer Orientation,1,1 pm,3:30 pm,qa5bogl8i12og8c52mjp890si8
2,09/09/2023,10 am - 2 pm,ec,ec,Kick-off,1,10 am,2 pm,b33uhacdr8glocr344f1egge3s
3,09/09/2023,10 am - 2 pm,WCT,wct-instructors,Kick-off,1,10 am,2 pm,f998sjl6ghr9muqj4fjrddqo54
4,09/09/2023,10 am - 2 pm,Math Matters,math-instructors,Kick-off,1,10 am,2 pm,ca6q7ju0qm13o28tntev47edao


<IPython.core.display.Javascript object>